### Comprehend Batch Processing

In order to scale your job loads, batch processing is very useful. For eg, if you have a load of 10,000 documents, its best to break in into 25 batches of 400 documents each. The example below demonstrates how to do batch job processing.

In [15]:
!rm -rf batch_data*
!mkdir batch_data

In [ ]:
region = 'us-east-1'
role_arn = 'arn:aws:iam::951145066533:role/service-role/AmazonComprehendServiceRole-_908203_cmpd'
model_arn = 'arn:aws:comprehend:us-east-1:951145066533:document-classifier/BBCNewsDataClassifier'
source_file = "Articles.csv"
bucket_prefix = 'batch_data'
bucket_name = 'comprehend-demo-20200602'

In [12]:
import os
import pandas as pd
import boto3
import json
import datetime

In [17]:
df = pd.DataFrame()
mapping = {}


In [50]:
colnames=['Article']
articles = pd.read_csv(source_file, encoding='cp1252',delimiter=',',header=0,names=colnames,usecols=[0,0])

In [51]:
print(articles.shape)
articles.head()

(2692, 1)


,Article
0,KARACHI: The Sindh government has decided to b...
1,HONG KONG: Asian markets started 2015 on an up...
2,HONG KONG: Hong Kong shares opened 0.66 perce...
3,HONG KONG: Asian markets tumbled Tuesday follo...
4,NEW YORK: US oil prices Monday slipped below $...


In [55]:
articles.to_csv('preprocessed_artices.csv',encoding='utf-8',header=None,index=False)

Breakdown the file into batches of 25

In [58]:
s3 = boto3.client('s3')
comprehend = boto3.client('comprehend')

In [60]:
# List all classifiers in account
list_response = comprehend.list_document_classifiers()
for item in list_response['DocumentClassifierPropertiesList']:
    eval_metrics = item['ClassifierMetadata']['EvaluationMetrics']
    print('{}:\n Precision {} Recall {} Accuracy {} F1 {}'.format(item['DocumentClassifierArn'],
                                                         eval_metrics['Precision'],
                                                         eval_metrics['Recall'],
                                                         eval_metrics['Accuracy'],
                                                         eval_metrics['F1Score']
                                                        )
         )

arn:aws:comprehend:us-east-1:951145066533:document-classifier/BBCSampleDataClassifier:
 Precision 0.9935 Recall 0.9933 Accuracy 0.9933 F1 0.9933
arn:aws:comprehend:us-east-1:951145066533:document-classifier/BBCTestDataClassifier:
 Precision 0.9735 Recall 0.9733 Accuracy 0.9733 F1 0.9732
arn:aws:comprehend:us-east-1:951145066533:document-classifier/BBCNewsDataClassifier:
 Precision 0.9541 Recall 0.9533 Accuracy 0.9533 F1 0.9532
arn:aws:comprehend:us-east-1:951145066533:document-classifier/BBCWorldNewsDataClassifier:
 Precision 0.9806 Recall 0.98 Accuracy 0.98 F1 0.98


In [61]:
model_arn='arn:aws:comprehend:us-east-1:951145066533:document-classifier/BBCSampleDataClassifier'